In [1]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import r2_score, accuracy_score, roc_curve, auc
import pandas as pd
import os
import numpy as np
from matplotlib import pyplot as plt
import random
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from scipy.stats import randint
from input_sampling import InputSampler

In [2]:
cat = ['Sex', 'Embarked']
num = ['Pclass', 'Age', 'Parch', 'Fare', 'SibSp']
train_set = pd.read_csv('train.csv')
train_set = train_set.fillna({
    'Pclass': train_set['Pclass'].mean(),
    'Age': train_set['Age'].mean(),
    'Parch': train_set['Parch'].mean(),
    'Fare': train_set['Fare'].mean(),
    'SibSp': train_set['SibSp'].mean()
})

y = train_set['Survived']
X = train_set[cat + num]

In [3]:
sampler = InputSampler(num, cat, train_set)

In [4]:
type(sampler.var_combinations[3][1])

list

In [5]:
sampler.sampleLogR()

[[0.8268156424581006, [['Pclass', 'Age', 'SibSp'], ['Sex']]],
 [0.8268156424581006, [['Pclass', 'Age', 'Parch'], ['Sex', 'Embarked']]],
 [0.8268156424581006, [['Pclass', 'Age', 'Fare', 'SibSp'], ['Sex']]],
 [0.8268156424581006, [['Pclass', 'Age', 'SibSp'], ['Sex', 'Embarked']]],
 [0.8268156424581006,
  [['Pclass', 'Age', 'Fare', 'SibSp'], ['Sex', 'Embarked']]],
 [0.8212290502793296, [['Pclass', 'Age', 'Parch'], ['Sex']]],
 [0.8212290502793296, [['Pclass', 'Age', 'Parch', 'Fare'], ['Sex']]],
 [0.8212290502793296, [['Pclass', 'Age', 'Parch', 'SibSp'], ['Sex']]],
 [0.8212290502793296, [['Pclass', 'Age', 'Parch', 'Fare', 'SibSp'], ['Sex']]],
 [0.8156424581005587,
  [['Pclass', 'Age', 'Parch', 'Fare'], ['Sex', 'Embarked']]],
 [0.8156424581005587,
  [['Pclass', 'Age', 'Parch', 'SibSp'], ['Sex', 'Embarked']]],
 [0.8100558659217877, [['Pclass', 'Age'], ['Sex']]],
 [0.8100558659217877, [['Pclass', 'Age', 'Fare'], ['Sex']]],
 [0.8100558659217877, [['Pclass', 'Age'], ['Sex', 'Embarked']]],
 [0.81

str